# ArcGIS Portal Usage Logger
## Description
Creates a usage log for an organization's ArcGIS Portal using user logins. The last login date for each user is initially recorded, with a new record being added for a user if the last login date is different from the lastest record (i.e., a more recent login). From this data, you can track what users are utilizing the organization's Portal the most and find any who rarely utilize it to aid with role reassignment. This data can also help inform Portal administrator on the potential impacts of a service outage on users.

This script is intented to act as an alternative to using ArcGIS Monitor. Ideally, this script should be scheduled to run every day and not done manually.
## Parameters
`csv_file` - A file path for the CSV file data will be stored in. This must end in `.csv`.

`log_fields` - A list of fields to log. Usernames and login dates are always logged. This parameter can be an empty string if no additional fields are required.

`save_daily_copy` - Whether or not to create a daily usage log file along with the composite CSV file. Daily files are named in the format: `{csv_file_name}_DAILY_{YYYYMMDD}.csv`.
## Notes
This script is built to use a login with ArcGIS Pro. To use another method, change the following line of code: `gis = GIS('pro')`.

In [1]:
#Params
##CSV File Path
csv_file = 'C:/path/to/your/file.csv'

##Data Fields to Log
log_fields = ['role'] 
#Options: 'udn', 'id', 'fullName', 'categories', 'emailStatus', 'firstName', 'lastName', 
#         'preferredView', 'description', 'email', 'userType', 'idpUsername', 'favGroupId', 
#         'mfaEnabled', 'storageUsage', 'storageQuota', 'orgId', 'level', 'userLicenseTypeId', 
#         'disabled', 'tags', 'culture', 'cultureFormat', 'region', 'units', 'thumbnail', 
#         'access', 'created', 'modified', 'provider', 'roleId', 'role'

#Whether to Save a Daily Copy of New Data
save_daily_copy = False

In [2]:
#Parameter Validation
if csv_file[-4::] != '.csv':
    raise ValueError('csv_file must be a file path ending with ".csv".')
if not isinstance(save_daily_copy, bool):
    raise ValueError('save_daily_copy must be a boolean value.')
if not isinstance(log_fields, list):
    raise ValueError('log_fields must be a list of values.')

In [3]:
#Import Modules
from datetime import datetime
from arcgis import GIS
import pandas as pd

In [4]:
#Read in Old Data
##If not found, create a new file
try:
    old_data = pd.read_csv(csv_file).to_dict(orient = 'records')
    old_data_exists = True
    
    #Get Latest Login Date
    ##Last login == last record in table for user
    last_login_dates = {}
    for record in reversed(old_data):
        if record['username'] not in last_login_dates:
            last_login_dates[record['username']] = record['lastLogin'].split(' ')[0]
    
except FileNotFoundError:
    old_data_exists = False

In [5]:
#Pull User Data
gis = GIS('pro')
users = gis.users.search(max_users = 10000)

#Iterate Through Users
log_data = []
for user in users:
    #Ignore Any Users who have Never Logged In
    if user['lastLogin'] == -1:
        continue
    
    #Get Date as a String
    date = str(datetime.fromtimestamp(int(user['lastLogin'] / 1000))).split(' ')[0]
    if date != str(datetime.today()).split(' ')[0]:
        continue
        
    #Check if the Date is Already Logged (Allows for Multiple Script Runs in a Single Day)
    if old_data_exists is True:
        if user['username'] in last_login_dates:
            if date == last_login_dates[user['username']]:
                continue

    #Collect Data and Add to New List
    user_data = {}
    user_data['username'] = user['username']
    user_data['lastLogin'] = date

    for field in log_fields:
        ##Format Date Fields
        if field in ['created','modified'] and user[field]:
            user_data[field] = str(datetime.fromtimestamp(int(user[field] / 1000)))
        else:
            user_data[field] = user[field]
            
    log_data.append(user_data)

#Combine Data and Convert to CSV
if save_daily_copy is True:
    date_today = datetime.strftime(datetime.now(), '20%y%m%d')
    daily_csv = f'{csv_file[:-4]}_DAILY_{date_today}.csv'
    pd.DataFrame(log_data).to_csv(daily_csv, index = False)
    
if old_data_exists is True:
    old_data.extend(log_data)
    pd.DataFrame(old_data).to_csv(csv_file, index = False)
else:
    pd.DataFrame(log_data).to_csv(csv_file, index = False)